In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function
from __future__ import absolute_import, unicode_literals
from IPython.display import display
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

import bokeh.io
import bokeh.mpl
import bokeh.plotting

%config InlineBackend.figure_formats = {'eps', 'retina'}
bokeh.io.output_notebook()

rc = {'lines.linewidth': 1, 
      'axes.labelsize': 14, 
      'axes.titlesize': 14, 
      'axes.facecolor': 'FFFFFF',
       }

sns.set_context('notebook', rc=rc)
sns.set_style('white', rc=rc)
sns.set_palette("colorblind")

pd.set_option('precision', 4)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Loading BokehJS ...

In [2]:
def is_bad(x):
    if x in [-0.5, -1.0, 9.5]:
        return True
    else:
        return False

def load_transform(fp):
    df = pd.read_json(fp)
    df = df.sort_values(by=['trip', 't'])
    bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    df['trip_interval'] = pd.cut(df['trip'], 
                                    bins=bins,
                                    precision=3,
                                    labels = trips_intervals)
    df['success'] = df['success'].astype(int)
    df['reward'] = df['reward'].astype(int)
    df['bad_moves'] = df['reward'].apply(lambda x: int(is_bad(x)))
    return df


def plot_path(df, deadline=False):
    df = df.sort_values(by=['trip', 't'])
    grid = sns.FacetGrid(df, col="trip", 
                         hue="trip",
                         col_wrap=5,
                         size=2.0,
                         aspect=1.5)
    grid.map(plt.axhline, y=0, ls=":", c=".5", label='No move')
    grid.map(plt.plot, "t", "reward", marker="o", ms=5, lw=.8)

    if deadline:
        grid.set(xticks=np.linspace(0, 50, 6, endpoint=True), xlim=(-1, 50), ylim=(-3, 13))
    else:
        grid.set(xticks=np.linspace(0, 150, 6, endpoint=True), xlim=(-1, 150), ylim=(-3, 13))
    grid.fig.tight_layout(w_pad=1)
    plt.savefig("./figures/random_agent.eps", format='eps', dpi=1000)

def review_results(df):
    print('Successful trips:')
    display(pd.crosstab(df.trip_interval, df.success, margins=True).T)
    
    print("\nRewards:")
    display(pd.crosstab(df.trip_interval, df.reward, margins=True).T)

    print("\nBad moves:")
    display(pd.crosstab(df.trip_interval, df.bad_moves, margins=True).T)
    
def publish_results(df):
    print(tabulate(df.groupby(['trip_interval'])['success', 'bad_moves'].sum(),
         headers=['Trips', 'Success', 'Bad moves'],
         tablefmt='latex_booktabs'))
    
trips_intervals = ['01 - 10', '11 - 20', '21 - 30', '31 - 40', '41 - 50', '51 - 60', '61 - 70', '71 - 80', '81 - 90', '91 - 100']

In [3]:
fp = "./results/random_agent.json"

In [4]:
df = load_transform(fp)
review_results(df)

Successful trips:


trip_interval,01 - 10,11 - 20,21 - 30,31 - 40,41 - 50,51 - 60,61 - 70,71 - 80,81 - 90,91 - 100,All
success,,,,,,,,,,,
0,979,618,830,635,816,804,957,522,577,522,7260
1,1,1,3,4,5,4,4,7,6,3,38
All,980,619,833,639,821,808,961,529,583,525,7298



Rewards:


trip_interval,01 - 10,11 - 20,21 - 30,31 - 40,41 - 50,51 - 60,61 - 70,71 - 80,81 - 90,91 - 100,All
reward,,,,,,,,,,,
-1,253,162,231,203,240,227,258,155,170,153,2052
0,559,348,439,332,453,422,533,264,333,281,3964
2,168,108,163,103,126,157,169,106,80,91,1271
9,0,1,0,0,1,0,0,2,0,0,4
12,0,0,0,1,1,2,1,2,0,0,7
All,980,619,833,639,821,808,961,529,583,525,7298



Bad moves:


trip_interval,01 - 10,11 - 20,21 - 30,31 - 40,41 - 50,51 - 60,61 - 70,71 - 80,81 - 90,91 - 100,All
bad_moves,,,,,,,,,,,
0,727,457,602,436,581,581,703,374,413,372,5246
1,253,162,231,203,240,227,258,155,170,153,2052
All,980,619,833,639,821,808,961,529,583,525,7298


In [5]:
publish_results(df)

\begin{tabular}{lrr}
\toprule
 Trips    &   Success &   Bad moves \\
\midrule
 01 - 10  &         1 &         253 \\
 11 - 20  &         1 &         162 \\
 21 - 30  &         3 &         231 \\
 31 - 40  &         4 &         203 \\
 41 - 50  &         5 &         240 \\
 51 - 60  &         4 &         227 \\
 61 - 70  &         4 &         258 \\
 71 - 80  &         7 &         155 \\
 81 - 90  &         6 &         170 \\
 91 - 100 &         3 &         153 \\
\bottomrule
\end{tabular}


In [6]:
first_20 = df[df['trip'] <= 20]
plot_path(first_20, deadline=False)

In [7]:
random_agent_deadline = load_transform("./results/random_agent_deadline.json")

In [8]:
publish_results(random_agent_deadline)

\begin{tabular}{lrr}
\toprule
 Trips    &   Success &   Bad moves \\
\midrule
 01 - 10  &         1 &          79 \\
 11 - 20  &         2 &          82 \\
 21 - 30  &         2 &          99 \\
 31 - 40  &         0 &          71 \\
 41 - 50  &         2 &          70 \\
 51 - 60  &         4 &          74 \\
 61 - 70  &         1 &          77 \\
 71 - 80  &         1 &          76 \\
 81 - 90  &         2 &          82 \\
 91 - 100 &         2 &          76 \\
\bottomrule
\end{tabular}
